In [3]:
import os
import json
import hashlib
import requests
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

In [19]:
BASE_URL = "https://educationdata.urban.org/api/v1/"
CACHE_DIR = "cache"
os.makedirs(CACHE_DIR, exist_ok=True)

YEARS = list(range(2003, 2021))
FIPS_LIST = [55, 27, 17, 18, 39, 19, 31, 8, 37, 26, 13, 41, 48]

RACE_MAP = {
    1: "White", 2: "Black", 3: "Hispanic", 4: "Asian",
    5: "American Indian/Alaska Native", 6: "Native Hawaiian/Pacific Islander",
    7: "Two or more races", 8: "Nonresident alien"
}

SEX_MAP = {
    1: "Male",
    2: "Female"
}

VALID_RACES = set(RACE_MAP.keys())
VALID_SEXES = set(SEX_MAP.keys())

city_state_to_county = {
    ("madison", "WI"): "Dane County, WI",
    ("st. paul", "MN"): "Ramsey County, MN",
    ("springfield", "IL"): "Sangamon County, IL",
    ("indianapolis", "IN"): "Marion County, IN",
    ("columbus", "OH"): "Franklin County, OH",
    ("des moines", "IA"): "Polk County, IA",
    ("lincoln", "NE"): "Lancaster County, NE",
    ("boulder", "CO"): "Boulder County, CO",
    ("raleigh", "NC"): "Wake County, NC",
    ("champaign", "IL"): "Champaign County, IL",
    ("green bay", "WI"): "Brown County, WI",
    ("milwaukee", "WI"): "Milwaukee County, WI",
    ("ann arbor", "MI"): "Washtenaw County, MI",
    ("athens", "GA"): "Clarke County, GA",
    ("eugene", "OR"): "Lane County, OR",
    ("rockwall", "TX"): "Rockwall County, TX"
}

target_counties = {
    "Dane County, WI", "Ramsey County, MN", "Sangamon County, IL", "Marion County, IN",
    "Franklin County, OH", "Polk County, IA", "Lancaster County, NE", "Boulder County, CO",
    "Wake County, NC", "Champaign County, IL", "Brown County, WI", "Milwaukee County, WI",
    "Washtenaw County, MI", "Clarke County, GA", "Lane County, OR", "Rockwall County, TX"
}

In [ ]:
unitid_county_map = []

for year in YEARS:
    url = f"{BASE_URL}college-university/ipeds/directory/{year}/"
    print(f"Fetching {year}...")

    try:
        response = requests.get(url)
        response.raise_for_status()
        results = response.json().get("results", [])

        for record in results:
            unitid = record.get("unitid")
            city = record.get("city", "").lower().strip()
            state = record.get("state_abbr", "").strip()
            county_full = city_state_to_county.get((city, state))

            if county_full and unitid:
                unitid_county_map.append({
                    "unitid": unitid,
                    "county_full": county_full,
                    "year": year
                })

    except Exception as e:
        print(f"Failed for {year}: {e}")

# Build DataFrame from results
df_mapping = pd.DataFrame(unitid_county_map)

# Deduplicate on unitid (keep latest)
df_mapping = df_mapping.sort_values("year").drop_duplicates(subset="unitid", keep="last")

Fetching 2000...
Fetching 2001...
Fetching 2002...
Fetching 2003...
Fetching 2004...
Fetching 2005...
Fetching 2006...
Fetching 2007...
Fetching 2008...
Fetching 2009...
Fetching 2010...
Fetching 2011...
Fetching 2012...
Fetching 2013...
Fetching 2014...
Fetching 2015...
Fetching 2016...
Fetching 2017...
Fetching 2018...
Fetching 2019...
Fetching 2020...
Fetching 2021...
Fetching 2022...
Fetching 2023...


In [28]:
unitid_map = df_mapping[['unitid', 'county_full']]


def cached_get_with_retry(url, retries=3, delay=1):
    hash_key = hashlib.md5(url.encode()).hexdigest()
    cache_path = os.path.join(CACHE_DIR, f"{hash_key}.json")

    if os.path.exists(cache_path):
        with open(cache_path, "r") as f:
            return json.load(f)

    for attempt in range(retries):
        try:
            resp = requests.get(url)
            resp.raise_for_status()
            data = resp.json()
            with open(cache_path, "w") as f:
                json.dump(data, f)
            return data
        except Exception as e:
            print(f"Error fetching {url} (attempt {attempt+1}): {e}")
            time.sleep(delay * (2 ** attempt))  # exponential backoff

    print(f"Failed to fetch after {retries} attempts: {url}")
    return None


def build_url(endpoint_template, year, fips):
    return f"{BASE_URL}{endpoint_template.format(year=year, fips=fips)}"


def fetch_url(url, data_type, value_field):
    data = cached_get_with_retry(url)
    if not data:
        return []

    results = []
    if (len(data.get("results")) > 0):
        for row in data.get("results"):
            unitid = row.get("unitid")
            #race = row.get("race")
            #sex = row.get("sex")
            value = row.get(value_field)
            fips = row.get("fips")

            if (
                unitid is not None and
                value is not None and
                value not in (-1, -2, -3) #and
                #race in VALID_RACES and
                #sex in VALID_SEXES
            ):
                results.append({
                    "unitid": unitid,
                    #"race": race,
                    #"sex": sex,
                    data_type: value * 100 if data_type == "graduation_rate" or data_type == "fall_retention" else value,
                    "year": row.get("year"),
                    "fips": fips
                })

    return results


def parallel_fetch(data_type, value_field, endpoint_template):
    all_urls = [build_url(endpoint_template, year, fips) for year in YEARS for fips in FIPS_LIST]
    records = []

    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(fetch_url, url, data_type, value_field) for url in all_urls]

        for future in as_completed(futures):
            try:
                records.extend(future.result())
            except Exception as e:
                print(f"Thread error: {e}")

    df = pd.DataFrame(records)
    df = df.merge(unitid_map, on="unitid", how="inner")
    #df['race'] = df['race'].map(RACE_MAP)
    #df['sex'] = df['sex'].map(SEX_MAP)

    return df

In [8]:
# Fetch enrollment numbers
enroll_df = parallel_fetch(
    data_type="enrollment",
    value_field="enrollment_fall",
    endpoint_template="college-university/ipeds/fall-enrollment/{year}/undergraduate/race/sex/?fips={fips}"
)

# Aggregate
agg_enroll = enroll_df.groupby(['county_full', 'race', 'sex', 'year'])['enrollment'].sum().reset_index()


Error fetching https://educationdata.urban.org/api/v1/college-university/ipeds/fall-enrollment/2021/undergraduate/race/sex/?fips=55 (attempt 1): 500 Server Error: Internal Server Error for url: https://educationdata.urban.org/api/v1/college-university/ipeds/fall-enrollment/2021/undergraduate/race/sex/?fips=55
Error fetching https://educationdata.urban.org/api/v1/college-university/ipeds/fall-enrollment/2021/undergraduate/race/sex/?fips=17 (attempt 1): 500 Server Error: Internal Server Error for url: https://educationdata.urban.org/api/v1/college-university/ipeds/fall-enrollment/2021/undergraduate/race/sex/?fips=17
Error fetching https://educationdata.urban.org/api/v1/college-university/ipeds/fall-enrollment/2021/undergraduate/race/sex/?fips=39 (attempt 1): 500 Server Error: Internal Server Error for url: https://educationdata.urban.org/api/v1/college-university/ipeds/fall-enrollment/2021/undergraduate/race/sex/?fips=39
Error fetching https://educationdata.urban.org/api/v1/college-unive

In [ ]:
# Fetch graduation rate for colleges - Unable to disagg by gender and race for this one
grad_df = parallel_fetch(
    data_type="graduation_rate",
    value_field="completion_rate_150pct",
    endpoint_template="college-university/ipeds/grad-rates/{year}?fips={fips}"
)

agg_grad = grad_df.groupby(
    ['county_full', 'race', 'sex', 'year']
)['graduation_rate'].mean().reset_index()

In [11]:
agg_grad

,county_full,race,sex,year,graduation_rate
0,"Boulder County, CO",American Indian/Alaska Native,Female,2000,38.500000
1,"Boulder County, CO",American Indian/Alaska Native,Female,2001,78.950000
2,"Boulder County, CO",American Indian/Alaska Native,Female,2002,68.750000
3,"Boulder County, CO",American Indian/Alaska Native,Female,2003,61.900000
4,"Boulder County, CO",American Indian/Alaska Native,Female,2004,80.566667
...,...,...,...,...,...
3576,"Washtenaw County, MI",White,Male,2013,45.100000
3577,"Washtenaw County, MI",White,Male,2014,44.837500
3578,"Washtenaw County, MI",White,Male,2015,47.075000
3579,"Washtenaw County, MI",White,Male,2016,47.840000


In [14]:
agg_enroll

,county_full,race,sex,year,enrollment
0,"Boulder County, CO",American Indian/Alaska Native,Female,2000,200
1,"Boulder County, CO",American Indian/Alaska Native,Female,2001,228
2,"Boulder County, CO",American Indian/Alaska Native,Female,2002,252
3,"Boulder County, CO",American Indian/Alaska Native,Female,2003,272
4,"Boulder County, CO",American Indian/Alaska Native,Female,2004,314
...,...,...,...,...,...
3096,"Washtenaw County, MI",White,Male,2016,36757
3097,"Washtenaw County, MI",White,Male,2017,37409
3098,"Washtenaw County, MI",White,Male,2018,37131
3099,"Washtenaw County, MI",White,Male,2019,36440


In [29]:
# Now lets fetch college retention rates
fall_retention_df = parallel_fetch(
    data_type="fall_retention",
    value_field="retention_rate",
    endpoint_template="college-university/ipeds/fall-retention/{year}?fips={fips}"
)

In [33]:
fall_retention_df
agg_fall_retention = fall_retention_df.groupby(
    ['county_full', 'year']
)['fall_retention'].mean().reset_index()

In [ ]:
agg_fall_retention #output

,county_full,year,fall_retention
0,"Boulder County, CO",2003,64.333333
1,"Boulder County, CO",2004,56.000000
2,"Boulder County, CO",2005,74.000000
3,"Boulder County, CO",2006,57.500000
4,"Boulder County, CO",2007,70.545455
...,...,...,...
265,"Washtenaw County, MI",2016,63.714286
266,"Washtenaw County, MI",2017,71.692308
267,"Washtenaw County, MI",2018,68.923077
268,"Washtenaw County, MI",2019,75.214286
